# AmericaSave Mortgage Assessment

## Import and connect to the database

In [1]:
import sqlite3

In [2]:
# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

In [3]:
%load_ext sql
%sql sqlite:///my_database.db

Connecting to 'sqlite:///my_database.db'

In [4]:
# Delete all tables from the database. This is useful when rerunning the notebook.

In [5]:
# Get all user-defined tables (excluding internal SQLite tables)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';")
tables = cursor.fetchall()

# Drop each table
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table[0]};")

In [6]:
# Read the tables
table_list = ['assignment_history SQL Table.txt', 'bookings SQL Table.txt', 'space_travel_agents SQL Table.txt']

for t in table_list:
    with open(t, 'r') as f:
        script = f.read()
    cursor.executescript(script)

In [7]:
## Merge table bookings and assignment_history so that bookings have information of AgentID

In [8]:
%%sql

CREATE TABLE bookings_2 AS
SELECT B.*,
        AH.AgentID
FROM bookings AS B
LEFT JOIN assignment_history AS AH USING(AssignmentID)

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Update loads and agent_ranking

In [9]:
%%sql 

ALTER TABLE space_travel_agents
ADD COLUMN load INTEGER DEFAULT 0;

ALTER TABLE space_travel_agents
ADD COLUMN agent_rank INTEGER;

Running query in 'sqlite:///my_database.db'

++
||
++
++

In [10]:
%%sql
UPDATE space_travel_agents
SET load = load + (
  SELECT COUNT(*)
  FROM bookings_2
  WHERE bookings_2.AgentID = space_travel_agents.AgentID
    AND bookings_2.BookingStatus = 'Pending'
)

Running query in 'sqlite:///my_database.db'

30 rows affected.

++
||
++
++

In [11]:
%%sql

UPDATE space_travel_agents
SET agent_rank = (
  SELECT COUNT(*)
  FROM space_travel_agents AS b
  WHERE b.load < space_travel_agents.load
     OR (b.load = space_travel_agents.load AND b.YearsOfService > space_travel_agents.YearsOfService)
     OR (b.load = space_travel_agents.load AND b.YearsOfService = space_travel_agents.YearsOfService AND b.AverageCustomerServiceRating > space_travel_agents.AverageCustomerServiceRating)
) + 1;

Running query in 'sqlite:///my_database.db'

30 rows affected.

++
||
++
++

In [12]:
%sql SELECT * FROM space_travel_agents LIMIT 5

Running query in 'sqlite:///my_database.db'

AgentID,FirstName,LastName,Email,JobTitle,DepartmentName,ManagerName,SpaceLicenseNumber,YearsOfService,AverageCustomerServiceRating,load,agent_rank
1,Aurora,Tanaka,aurora.tanaka@astra2081.com,Senior Space Travel Agent,Interplanetary Sales,Lyra Chen,SL-2081-001,12,4.0,0,5
2,Kai,Rodriguez,kai.rodriguez@astra2081.com,Space Travel Agent,Luxury Voyages,Lyra Chen,SL-2081-002,7,4.0,1,21
3,Nova,Singh,nova.singh@astra2081.com,Senior Space Travel Agent,Premium Bookings,Zane Holloway,SL-2081-003,15,4.0,0,2
4,Leo,Kim,leo.kim@astra2081.com,Space Travel Agent,Interplanetary Sales,Lyra Chen,SL-2081-004,5,4.0,0,10
5,Vera,Nguyen,vera.nguyen@astra2081.com,Space Travel Agent,Luxury Voyages,Zane Holloway,SL-2081-005,3,3.9,1,24


## Create agent_rank_tracker

In [13]:
%%sql

CREATE TABLE IF NOT EXISTS agent_rank_tracker (
    AgentID INTEGER PRIMARY KEY,
    agent_rank INTEGER 
);

INSERT INTO agent_rank_tracker (AgentID, agent_rank)
    SELECT a.AgentID,
        (
            SELECT COUNT(*)
            FROM space_travel_agents AS b
            WHERE b.load < a.load
              OR (b.load = a.load AND b.YearsOfService > a.YearsOfService)
              OR (b.load = a.load AND b.YearsOfService = a.YearsOfService AND b.AverageCustomerServiceRating > a.AverageCustomerServiceRating)
         ) + 1
    FROM space_travel_agents AS a;

Running query in 'sqlite:///my_database.db'

30 rows affected.

++
||
++
++

In [14]:
%sql SELECT * FROM agent_rank_tracker ORDER BY agent_rank LIMIT 5

Running query in 'sqlite:///my_database.db'

AgentID,agent_rank
6,1
3,2
27,2
24,4
1,5


## Add a new table, named new_customer

In [15]:
%%sql
CREATE TABLE new_customer (
    CustomerName VARCHAR(100) PRIMARY KEY,
    CommunicationMethod VARCHAR(50) NOT NULL CHECK (CommunicationMethod IN ('Phone Call', 'Text')),
    LeadSource VARCHAR(50) NOT NULL CHECK (LeadSource IN ('Organic', 'Bought')),
    Destination VARCHAR(50) NOT NULL CHECK (Destination IN ('Mars', 'Europa', 'Titan', 'Venus', 'Ganymede')),
    LaunchLocation VARCHAR(200) NOT NULL CHECK (LaunchLocation IN ('Dallas-Fort Worth Launch Complex', 
    'New York Orbital Gateway', 'Dubai Interplanetary Hub',
    'Tokyo Spaceport Terminal', 'London Ascension Platform', 'Sydney Stellar Port'))
    )

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Step 1: Update assignment_history

In [16]:
%%sql
CREATE TRIGGER updating_assignment_history
AFTER INSERT ON new_customer
FOR EACH ROW
BEGIN

    -- Insert assignment_history
    INSERT INTO assignment_history (
                CustomerName,
                AssignmentID,
                CommunicationMethod,
                LeadSource,
                AssignedDateTime,
                AgentID
                )
    VALUES (
            NEW.CustomerName,
            (SELECT IFNULL(MAX(AssignmentID), 0) + 1 FROM assignment_history),
                    NEW.CommunicationMethod,
                    NEW.LeadSource,
                    datetime('now', '+56 years'),
            (SELECT AgentID FROM agent_rank_tracker ORDER BY agent_rank LIMIT 1)
    );
END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Step 2: Insert into bookings_2

In [17]:
%%sql
CREATE TRIGGER updating_bookings_2
AFTER INSERT ON assignment_history
FOR EACH ROW
BEGIN
    
    INSERT INTO bookings_2 (
                BookingID,
                AssignmentID,
                Destination,
                LaunchLocation,
                BookingStatus,
                AgentID
                )
    VALUES (
            (SELECT IFNULL(MAX(BookingID), 0) + 1 FROM bookings_2),
                    NEW.AssignmentID,
                    (SELECT Destination FROM new_customer WHERE CustomerName = NEW.CustomerName),
                    (SELECT LaunchLocation FROM new_customer WHERE CustomerName = NEW.CustomerName),
                    'Pending',
                    NEW.AgentID
            );
END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Step 3a: Updating load if new row is added to table bookings_2

In [18]:
%%sql

CREATE TRIGGER updating_loads
AFTER INSERT ON bookings_2
FOR EACH ROW
BEGIN
    UPDATE space_travel_agents
    SET load = load + 1
    WHERE AgentID = NEW.AgentID AND NEW.BookingStatus = 'Pending';
END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Step 3b: In case the BookingStatus changed from "Pending" to "Cancelled" or "Confirmed"

In [19]:
%%sql

CREATE TRIGGER update_loads_subtracting
AFTER UPDATE OF BookingStatus ON bookings_2
FOR EACH ROW
    WHEN OLD.BookingStatus = 'Pending' AND NEW.BookingStatus IN ('Confirmed', 'Cancelled')
    BEGIN
        UPDATE space_travel_agents
        SET load = MAX(load - 1, 0)
        WHERE AgentID = NEW.AgentID;
    END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

## Step 4: Recomputing agent_rank

In [20]:
%%sql
    
CREATE TRIGGER recompute_agent_rank_on_load
AFTER UPDATE OF load ON space_travel_agents
FOR EACH ROW
BEGIN
    DELETE FROM agent_rank_tracker;

    INSERT INTO agent_rank_tracker (AgentID, agent_rank)
    SELECT a.AgentID,
            (SELECT COUNT(*)
                FROM space_travel_agents AS b
                WHERE b.load < a.load
                OR (b.load = a.load AND b.YearsOfService > a.YearsOfService)
                OR (b.load = a.load AND b.YearsOfService = a.YearsOfService
                AND b.AverageCustomerServiceRating > a.AverageCustomerServiceRating)
            ) + 1
FROM space_travel_agents AS a;
END;

Running query in 'sqlite:///my_database.db'

++
||
++
++

## If BookingStatus changes from 'Pending' to 'Cancelled' or 'Confirmed'

## Test run

## 1. Insert new rows to "new_customer"

Before inserting new rows, here is my agent_ranking_tracker:

In [21]:
%sql SELECT * FROM agent_rank_tracker ORDER BY agent_rank LIMIT 5

Running query in 'sqlite:///my_database.db'

AgentID,agent_rank
6,1
3,2
27,2
24,4
1,5


In [22]:
%%sql

INSERT INTO new_customer (
                        CustomerName, 
                        CommunicationMethod,
                        LeadSource, 
                        Destination, 
                        LaunchLocation
                        )
VALUES (
        'Kate Nguyen', 
        'Text', 
        'Organic', 
        'Mars', 
        'Dallas-Fort Worth Launch Complex'),
        (
        'Jonh Doe', 
        'Phone Call', 
        'Bought', 
        'Europa', 
        'Sydney Stellar Port'
        )

Running query in 'sqlite:///my_database.db'

2 rows affected.

++
||
++
++

In [23]:
%sql SELECT * FROM assignment_history ORDER BY AssignmentID DESC

Running query in 'sqlite:///my_database.db'

AssignmentID,AgentID,CustomerName,CommunicationMethod,LeadSource,AssignedDateTime
452,3,Jonh Doe,Phone Call,Bought,2081-07-10 21:48:36
451,6,Kate Nguyen,Text,Organic,2081-07-10 21:48:36
450,11,Mira Cruz,Phone Call,Bought,2081-04-10 15:00:00
449,20,Arlo King,Text,Organic,2081-04-10 13:50:00
448,2,Tinsley Ross,Text,Bought,2081-04-10 12:40:00
447,16,Elowen Bell,Phone Call,Organic,2081-04-10 11:30:00
446,24,Callahan Rivera,Text,Bought,2081-04-10 10:20:00
445,7,Kylan Scott,Text,Organic,2081-04-10 09:10:00
444,28,Zya Lewis,Phone Call,Bought,2081-04-09 14:55:00
443,13,Azalea Brooks,Text,Organic,2081-04-09 13:45:00


In [24]:
%sql SELECT * FROM agent_rank_tracker ORDER BY agent_rank LIMIT 10

Running query in 'sqlite:///my_database.db'

AgentID,agent_rank
27,1
24,2
1,3
16,3
19,5
11,6
25,7
4,8
23,9
29,10


## 2. If BookingStatus changed from "Pending" to "Cancelled"

Let's find the AgentID that has load > 1

In [25]:
%sql SELECT * FROM space_travel_agents WHERE load > 0 ORDER BY agent_rank LIMIT 5

Running query in 'sqlite:///my_database.db'

AgentID,FirstName,LastName,Email,JobTitle,DepartmentName,ManagerName,SpaceLicenseNumber,YearsOfService,AverageCustomerServiceRating,load,agent_rank
6,Soren,Baker,soren.baker@astra2081.com,Lead Space Travel Agent,Premium Bookings,Zane Holloway,SL-2081-006,18,4.6,1,1
3,Nova,Singh,nova.singh@astra2081.com,Senior Space Travel Agent,Premium Bookings,Zane Holloway,SL-2081-003,15,4.0,1,2
22,Juno,Brooks,juno.brooks@astra2081.com,Lead Space Travel Agent,Premium Bookings,Zane Holloway,SL-2081-022,17,4.8,1,17
12,Nia,Ramirez,nia.ramirez@astra2081.com,Senior Space Travel Agent,Premium Bookings,Zane Holloway,SL-2081-012,14,5.0,1,18
8,Orion,Davis,orion.davis@astra2081.com,Senior Space Travel Agent,Luxury Voyages,Zane Holloway,SL-2081-008,10,4.4,1,19


Let's choose AgentID = 14 with load = 2 and current rank = 26

In [26]:
%%sql
SELECT * 
FROM bookings_2
WHERE AgentID = 3 AND BookingStatus = 'Pending'

Running query in 'sqlite:///my_database.db'

BookingID,AssignmentID,BookingCompleteDate,CancelledDate,Destination,Package,LaunchLocation,DestinationRevenue,PackageRevenue,TotalRevenue,BookingStatus,AgentID
414,452,None,None,Europa,None,Sydney Stellar Port,None,None,None,Pending,3


Let's change the BookingStatus of BookingID = 414 into 'Cancelled'.

In [27]:
%%sql
UPDATE bookings_2
SET BookingStatus = 'Cancelled'
WHERE AgentID = 3 AND BookingID = 414;

SELECT * 
FROM bookings_2
WHERE AgentID = 3 AND BookingID = 414;

Running query in 'sqlite:///my_database.db'

1 rows affected.

BookingID,AssignmentID,BookingCompleteDate,CancelledDate,Destination,Package,LaunchLocation,DestinationRevenue,PackageRevenue,TotalRevenue,BookingStatus,AgentID
414,452,None,None,Europa,None,Sydney Stellar Port,None,None,None,Cancelled,3


Let's check the load

In [28]:
%sql SELECT * FROM space_travel_agents WHERE AgentID = 3

Running query in 'sqlite:///my_database.db'

AgentID,FirstName,LastName,Email,JobTitle,DepartmentName,ManagerName,SpaceLicenseNumber,YearsOfService,AverageCustomerServiceRating,load,agent_rank
3,Nova,Singh,nova.singh@astra2081.com,Senior Space Travel Agent,Premium Bookings,Zane Holloway,SL-2081-003,15,4.0,0,2


In [29]:
%sql SELECT * FROM agent_rank_tracker ORDER BY agent_rank LIMIT 5

Running query in 'sqlite:///my_database.db'

AgentID,agent_rank
3,1
27,1
24,3
1,4
16,4


AgentID = 3 becomes the first available as they don't have any loads. Our algorithms work for both cases beautifully!